In [ ]:
!pip install -q tifffile segmentation-models==1.0.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.6 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import random

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Model


In [ ]:
def load_and_normalize_tiff(image_path):
    image = tifffile.imread(image_path)

    if len(image.shape) == 3 and image.shape[0] == 12:
        image = np.transpose(image, (1, 2, 0))

    normalized = np.zeros_like(image, dtype=np.float32)
    for i in range(image.shape[2]):
        layer = image[:, :, i].astype(np.float32)
        if layer.max() > layer.min():
            normalized[:, :, i] = (layer - layer.min()) / (layer.max() - layer.min())
    return normalized

def load_mask(mask_path):
    mask = np.array(Image.open(mask_path))
    if len(mask.shape) == 3:
        mask = mask[:, :, 0]
    return mask


In [ ]:
def load_dataset(image_dir, mask_dir):
    image_paths = sorted(glob(os.path.join(image_dir, '*.tif')))
    mask_paths = sorted(glob(os.path.join(mask_dir, '*.png')))

    images = [load_and_normalize_tiff(p) for p in image_paths]
    masks = [load_mask(p) for p in mask_paths]

    return np.array(images), np.array(masks)


In [ ]:
def deeplabv3_plus_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    low_level_features = x

    x = layers.Conv2D(128, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # ASPP
    aspp1 = layers.Conv2D(256, 1, padding='same', activation='relu')(x)
    aspp2 = layers.Conv2D(256, 3, padding='same', dilation_rate=6, activation='relu')(x)
    aspp3 = layers.Conv2D(256, 3, padding='same', dilation_rate=12, activation='relu')(x)
    aspp4 = layers.Conv2D(256, 3, padding='same', dilation_rate=18, activation='relu')(x)

    avg = layers.GlobalAveragePooling2D()(x)
    avg = layers.Reshape((1, 1, x.shape[-1]))(avg)
    avg = layers.Conv2D(256, 1, activation='relu')(avg)
    avg = layers.UpSampling2D(size=(x.shape[1], x.shape[2]), interpolation='bilinear')(avg)

    x = layers.concatenate([aspp1, aspp2, aspp3, aspp4, avg])
    x = layers.Conv2D(256, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)

    # Decoder
    low = layers.Conv2D(48, 1, padding='same', activation='relu')(low_level_features)
    low = layers.BatchNormalization()(low)
    x = layers.concatenate([x, low])
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(x)

    return Model(inputs, outputs)


In [ ]:
import os
from glob import glob
import numpy as np
import tifffile
from PIL import Image

def load_images_and_masks(image_dir, mask_dir):
    image_paths = sorted(glob(os.path.join(image_dir, "*.tif")))
    mask_paths = sorted(glob(os.path.join(mask_dir, "*.png")))

    X = []
    Y = []

    image_names = [os.path.basename(p).replace(".tif", "") for p in image_paths]
    mask_dict = {os.path.basename(p).replace(".png", ""): p for p in mask_paths}

    for name in image_names:
        if name in mask_dict:
            img_path = os.path.join(image_dir, f"{name}.tif")
            mask_path = mask_dict[name]

            # Load and normalize image
            image = tifffile.imread(img_path)
            if image.shape[0] == 12:  # Channels first
                image = np.transpose(image, (1, 2, 0))
            image = image.astype(np.float32)
            for i in range(image.shape[-1]):
                layer = image[:, :, i]
                image[:, :, i] = (layer - layer.min()) / (layer.max() - layer.min() + 1e-5)
            X.append(image)

            # Load mask
            mask = np.array(Image.open(mask_path))
            if mask.ndim == 3:
                mask = mask[:, :, 0]
            Y.append(mask)

    return np.array(X), np.array(Y)


In [ ]:
X, Y = load_images_and_masks("/content/drive/MyDrive/AI/satalite data/data/images", "/content/drive/MyDrive/AI/satalite data/data/labels")

print("Loaded images:", X.shape)
print("Loaded masks:", Y.shape)


Loaded images: (306, 128, 128, 12)
Loaded masks: (306, 128, 128)


In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

num_classes = len(np.unique(Y))
Y_cat = to_categorical(Y, num_classes=num_classes)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y_cat, test_size=0.2, random_state=42)


In [ ]:
model = deeplabv3_plus_model(input_shape=(128, 128, 12), num_classes=num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=15, batch_size=8)
model.save("deeplab_model.keras")


Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 331s 10s/step - accuracy: 0.7026 - loss: 1.0884 - val_accuracy: 0.8433 - val_loss: 0.3861
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 320s 10s/step - accuracy: 0.8351 - loss: 0.4018 - val_accuracy: 0.7891 - val_loss: 0.4791
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 325s 10s/step - accuracy: 0.8705 - loss: 0.3303 - val_accuracy: 0.7756 - val_loss: 0.5155
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 377s 10s/step - accuracy: 0.8890 - loss: 0.2785 - val_accuracy: 0.7756 - val_loss: 0.4870
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 320s 10s/step - accuracy: 0.8755 - loss: 0.2875 - val_accuracy: 0.8003 - val_loss: 0.4115
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 323s 10s/step - accuracy: 0.9082 - loss: 0.2400 - val_accuracy: 0.7764 - val_loss: 0.4805
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 321s 10s/step - accuracy: 0.8980 - loss: 0.2610 - val_accuracy: 0.8742 - val_loss: 0.2835
Epoch 8/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 318s 10s/step - accuracy: 0.9099 - loss: 0.2222 - val_accuracy: 0.

In [2]:
!pip install flask flask-ngrok tifffile --quiet


In [18]:
def preprocess_tiff_image(path):
    image = tifffile.imread(path)
    if image.shape[0] == 12:  # (12, H, W)
        image = np.transpose(image, (1, 2, 0))
    image = image.astype(np.float32)
    for i in range(image.shape[-1]):
        layer = image[:, :, i]
        image[:, :, i] = (layer - layer.min()) / (layer.max() - layer.min() + 1e-5)
    return np.expand_dims(image, axis=0)  # shape: (1, H, W, 12)


In [19]:
def save_comparison(pred_mask, gt_mask, save_path="static/comparison.png"):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title("Ground Truth")
    plt.imshow(gt_mask, cmap="viridis")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Predicted Mask")
    plt.imshow(pred_mask, cmap="viridis")
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()


In [20]:
import matplotlib.pyplot as plt
import os

def save_prediction_mask(pred, save_path="static/prediction.png"):
    mask = np.argmax(pred, axis=-1)[0]  # shape: (H, W)
    plt.imsave(save_path, mask, cmap="viridis")


In [ ]:
from google.colab import files
files.download('deeplab_model.keras')  # أو .h5 لو استخدمت الصيغة القديمة


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
from google.colab import files
uploaded = files.upload()


Saving deeplab_model.keras to deeplab_model.keras


In [15]:
from tensorflow.keras.models import load_model
model = load_model("deeplab_model.keras", compile=False)


In [6]:
!pip install pyngrok


In [8]:
!ngrok config add-authtoken 30UXwr4SVptRIFWfBFlDaBbfx1w_6d6q4Gf7h1yHgPkyCEnFf


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [21]:
GROUND_TRUTH_MASK_DIR = "/content/drive/MyDrive/AI/satalite data/data/labels"


In [26]:
import os
print(os.listdir("/content/drive/MyDrive/AI/satalite data/data/labels"))


['45.png', '150.png', '102.png', '249.png', '118.png', '21.png', '93.png', '217.png', '41.png', '66.png', '183.png', '240.png', '77.png', '267.png', '127.png', '134.png', '245.png', '13.png', '186.png', '268.png', '16.png', '187.png', '116.png', '204.png', '278.png', '205.png', '59.png', '138.png', '302.png', '172.png', '198.png', '296.png', '57.png', '60.png', '106.png', '137.png', '192.png', '62.png', '223.png', '236.png', '126.png', '272.png', '276.png', '113.png', '260.png', '135.png', '287.png', '56.png', '68.png', '105.png', '114.png', '90.png', '14.png', '42.png', '49.png', '202.png', '293.png', '142.png', '1.png', '119.png', '225.png', '81.png', '256.png', '242.png', '182.png', '254.png', '303.png', '103.png', '234.png', '84.png', '140.png', '18.png', '194.png', '301.png', '159.png', '10.png', '163.png', '261.png', '179.png', '51.png', '110.png', '29.png', '185.png', '165.png', '201.png', '44.png', '2.png', '75.png', '221.png', '258.png', '299.png', '257.png', '52.png', '176.pn

In [29]:
import os
os.makedirs("static", exist_ok=True)

In [ ]:
from flask import Flask, request, render_template_string, send_file
from pyngrok import ngrok
import numpy as np
import tifffile
from PIL import Image
import matplotlib.pyplot as plt
import io

from flask import Flask
from tensorflow.keras.models import load_model

model = load_model("deeplab_model.keras", compile=False)

GROUND_TRUTH_MASK_DIR = "/content/drive/MyDrive/AI/satalite data/data/labels"

# إنشاء Flask app
app = Flask(__name__)
public_url = ngrok.connect(5000)
print("🚀 Flask app running at:", public_url)

# HTML واجهة بسيطة
html = """
<!doctype html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>🛰️ DeepLabV3+ Segmentation</title>
  <style>
    body {
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      background: linear-gradient(to right, #1e3c72, #2a5298);
      color: #fff;
      text-align: center;
      padding-top: 50px;
    }

    h2 {
      font-size: 28px;
      margin-bottom: 20px;
    }

    form {
      background-color: rgba(255, 255, 255, 0.1);
      display: inline-block;
      padding: 30px;
      border-radius: 20px;
      box-shadow: 0 8px 16px rgba(0,0,0,0.3);
    }

    input[type="file"] {
      font-size: 16px;
      padding: 10px;
      border-radius: 8px;
      margin-bottom: 15px;
    }

    input[type="submit"] {
      font-size: 18px;
      background-color: #00c3ff;
      border: none;
      padding: 10px 20px;
      color: white;
      border-radius: 8px;
      cursor: pointer;
      transition: background 0.3s ease;
    }

    input[type="submit"]:hover {
      background-color: #0090c7;
    }

    .result-img {
    margin-top: 30px;
    border: 5px solid white;
    border-radius: 12px;
    box-shadow: 0 5px 15px rgba(0,0,0,0.5);
    transition: transform 0.3s ease;
    }
    .result-img:hover {
    transform: scale(1.05);
    }
  </style>
</head>
<body>
  <h2>Upload a 🛰️ Satellite Image (.tif) for Segmentation</h2>
  <form method=post enctype=multipart/form-data>
    <input type=file name=file><br><br>
    <input type=submit value="Predict Mask">
  </form>

  {% if result %}
   <div>
    <h3>🛰️ Original Satellite Image:</h3>
    <img src="{{ url_for('static', filename='input_preview.png') }}" width="512" class="result-img">
   </div>
   <div>
    <h3>📊 Predicted vs Ground Truth:</h3>
    <img src="{{ url_for('static', filename='comparison.png') }}" width="512" class="result-img">
   </div>
  {% endif %}


</body>
</html>
"""


# دالة normalizing نفس اللي استخدمتها للتدريب
def normalize_tiff_image(image):
    if len(image.shape) == 3 and image.shape[0] == 12:
        image = np.transpose(image, (1, 2, 0))

    norm_image = np.zeros_like(image, dtype=np.float32)
    for i in range(image.shape[2]):
        band = image[:, :, i].astype(np.float32)
        min_val = np.min(band)
        max_val = np.max(band)
        norm_image[:, :, i] = (band - min_val) / (max_val - min_val + 1e-6)
    return norm_image


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        tif_file = request.files['file']

        if tif_file.filename.endswith('.tif'):
            # 🧠 استخرج اسم الملف بدون الامتداد
            base_name = os.path.splitext(tif_file.filename)[0]

            # 🗂️ ابحث عن الماسك الحقيقي من نفس الاسم
            gt_path = os.path.join(GROUND_TRUTH_MASK_DIR, base_name + ".png")

            print(f"📂 Uploaded filename: {tif_file.filename}")
            print(f"🔍 Expected GT mask path: {gt_path}")
            print(f"🗂️ Mask directory exists? {os.path.exists(GROUND_TRUTH_MASK_DIR)}")


            if not os.path.exists(gt_path):
                return f"❌ Ground truth mask not found for {base_name}.png", 404

            # ✅ اقرأ الصورة .tif وطبّق normalization
            img = tifffile.imread(tif_file)
            img = normalize_tiff_image(img)

            # تحقق من الحجم والشكل
            from skimage.transform import resize
            if img.shape[:2] != (128, 128):
                img = resize(img, (128, 128, img.shape[2]), preserve_range=True, anti_aliasing=True)

            if img.shape[-1] != 12:
                return "❌ Error: Input image must have 12 channels", 400

            img = np.expand_dims(img, axis=0)

            # 🔮 توقع الماسك
            pred = model.predict(img)
            pred_mask = np.argmax(pred[0], axis=-1)

            # ⬅️ حفظ صورة input الأصلية (أول 3 قنوات)
            input_vis = img[0][:, :, :3]
            plt.imsave("static/input_preview.png", input_vis)


            # 📥 حمّل الماسك الحقيقي
            gt_mask = np.array(Image.open(gt_path))
            print(f"✅ Loaded GT mask with shape: {gt_mask.shape}")

            if gt_mask.ndim == 3:
                gt_mask = gt_mask[:, :, 0]

            # تأكد من تطابق الحجم
            if gt_mask.shape != pred_mask.shape:
                gt_mask = resize(gt_mask, pred_mask.shape, preserve_range=True, anti_aliasing=False).astype(np.uint8)

            # 🔄 احفظ مقارنة
            save_comparison(pred_mask, gt_mask)

            return render_template_string(html, result=True)

    return render_template_string(html)



@app.route('/predict')
def predict():
    return send_file("result.png", mimetype='image/png')

# تشغيل التطبيق
app.run(port=5000)


🚀 Flask app running at: NgrokTunnel: "https://d96cb6feceda.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Jul/2025 08:05:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2025 08:05:25] "GET /favicon.ico HTTP/1.1" 404 -


📂 Uploaded filename: 161.tif
🔍 Expected GT mask path: /content/drive/MyDrive/AI/satalite data/data/labels/161.png
🗂️ Mask directory exists? True
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 706ms/step


INFO:werkzeug:127.0.0.1 - - [31/Jul/2025 08:05:35] "POST / HTTP/1.1" 200 -


✅ Loaded GT mask with shape: (128, 128)


INFO:werkzeug:127.0.0.1 - - [31/Jul/2025 08:05:35] "GET /static/comparison.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2025 08:05:35] "GET /static/input_preview.png HTTP/1.1" 200 -
